# Assignment 6 (+ optional assignment) (due Dec 13th at 11:59pm)

> **NOTE:** Although the assignment is due on Dec 13th, Gradescope will remain open and accepting late submissions with no penalties until Dec 20th at midnight.

- This assignment covers SoftMax. Please refer to the class notes and corresponding Colabs on the course website for the required background.

- This assignment is worth **200 points**. Half of the points are for Assignment 6 and the other half for the the **optional** assignment. After completing the solutions **you must submit a copy of this notebook (`.ipynb`), including all your answers**.

- You can work alone or in pairs.  At the time of submission please **ONLY SUBMIT ONE FILE PER TEAM**.  Make sure to add all team members in your Gradescope submission.

- You can't use `scikit-learn` for answering questions on this assignment.

>  **Important**: Make sure **ALL cells are executed** before saving/downloading a copy of the notebook you will submit.

> **Credit:** The starter code in this assignment is an adaptation of one of Stanford's CS231n assignments.

---
## ALL ANSWERS must be entered ONLY using the indicated spaces marked as TODOs, please DO NOT modify any of the starter code.
> you must replace `pass` with your own code
---



In [ ]:
#####################################################################
# TODO: 0 points                                                    #
# Type your full names, using comments, one per line                #
#####################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
#Joseph Reyes
#Islam Dymov
# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

In [ ]:
###################################################
# do not modify this cell
###################################################

# imports and configuration
import time
from random import randrange
from keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# define constants for the size of all dataset partitions
n_train = 45000
n_valid = 1000
n_test = 1000
n_dev = 200

In [ ]:
###################################################
# do not modify this cell
###################################################

# downloading the CIFAR10 dataset
(Xtr, Ytr), (Xte, Yte) = cifar10.load_data()

# subsample the data for more efficient execution
mask = list(range(n_train, n_train + n_valid))
Xval, Yval = Xtr[mask], Ytr[mask]
mask = list(range(n_train))
Xtr, Ytr = Xtr[mask], Ytr[mask]
mask = list(range(n_test))
Xte, Yte = Xte[mask], Yte[mask]
mask = np.random.choice(n_train, n_dev, replace=False)
Xdev, Ydev = Xtr[mask], Ytr[mask]

# reshaping images into flat rows
Xtr = np.reshape(Xtr, (Xtr.shape[0], -1))
Xval = np.reshape(Xval, (Xval.shape[0], -1))
Xte = np.reshape(Xte, (Xte.shape[0], -1))
Xdev = np.reshape(Xdev, (Xdev.shape[0], -1))

# normalize the data: subtracting the mean image
mean = np.mean(Xtr, axis = 0)
Xtr = Xtr.astype(float, casting='safe', copy=False) - mean
Xval = Xval.astype(float, casting='safe', copy=False) - mean
Xte = Xte.astype(float, casting='safe', copy=False) - mean
Xdev = Xdev.astype(float, casting='safe', copy=False) - mean

# prepend column of 1s (bias)
Xtr = np.concatenate((np.ones((Xtr.shape[0],1)),Xtr), axis=1)
Xval = np.concatenate((np.ones((Xval.shape[0],1)),Xval), axis=1)
Xte = np.concatenate((np.ones((Xte.shape[0],1)),Xte), axis=1)
Xdev = np.concatenate((np.ones((Xdev.shape[0],1)),Xdev), axis=1)

# sanity check
print(f'Training data shapes:\timages {Xtr.shape} labels {Ytr.shape}')
print(f'Validation data shapes:\timages {Xval.shape} labels {Yval.shape}')
print(f'Test data shapes:\timages {Xte.shape} labels {Yte.shape}')
print(f'Development data shapes:\timages {Xdev.shape} labels {Ydev.shape}')

170498071/170498071 [==============================] - 5s 0us/step
Training data shapes:	images (45000, 3073) labels (45000, 1)
Validation data shapes:	images (1000, 3073) labels (1000, 1)
Test data shapes:	images (1000, 3073) labels (1000, 1)
Development data shapes:	images (200, 3073) labels (200, 1)


In [ ]:
def softmax_loss_naive(W, X, Y, reg):
    """
    Softmax loss function, naive implementation (with loops)
    Observations have dimension D, there are C classes, and we operate on 
    minibatches of N examples.

    Inputs:
    - W: A numpy array of shape (D, C) containing weights.
    - X: A numpy array of shape (N, D) containing a minibatch of data.
    - y: A numpy array of shape (N,) containing training labels; y[i] = c means
      that X[i] has label c, where 0 <= c < C.
    - lambda: regularization strength

    Returns a tuple of:
    - cross-entropy loss as single float
    - gradient with respect to weights W; an array of same shape as W
    """
    # initialize the loss and gradient to zero.
    loss = 0.0
    dW = np.zeros(W.shape)
    #############################################################################
    # TODO: 50 points (assignment 6)                                            #
    # Compute the softmax loss and its gradient using explicit loops.           #
    # Store the loss in loss and the gradient in dW. If you are not careful     #
    # here, it is easy to run into numeric instability. Don't forget to         #
    # implement the L2 regularization!                                          #
    #############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    #need to get the dimensions we will be working with
    classes = W.shape[1]
    train = X.shape[0]
    dimensions = W.shape[0]

    for i in range(train):
      score_list = X[i,:].dot(W)
      #scores_log = np.exp(-score_list.max()) #normalize for stability
      scores_e = np.exp(score_list)
      SoS = np.sum(scores_e)
      score_probability = scores_e/SoS 
      #loss+= -np.log(score_probability[Y[i]])

      for j in range(dimensions):
          for k in range (classes):
              if k == Y[i]:
                dW[j,k] += X.T[j,i]*(score_probability[k] - 1)
          
              else:
                dW[j,k] +=X.T[j,i] * score_probability[k]

      loss+= -np.log(score_probability[Y[i]])
    
    loss /=train #divide regularized term by m
    loss += 0.5*reg*np.sum(W**2) #divide by 2 and square

    #update dW
    dW /= train
    dW += reg*W 


    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    return loss, dW


def softmax_loss_vectorized(W, X, y, reg):
    """
    Inputs and outputs are the same as softmax_loss_naive.
    """
    # initialize the loss and gradient to zero.
    loss = 0.0
    dW = np.zeros(W.shape)
    #############################################################################
    # TODO: 50 points (assignment 6)                                            #
    # Compute the softmax loss and its gradient using no explicit loops.        #
    # Store the loss in loss and the gradient in dW.  If you are not careful    #
    # here, it is easy to run into numeric instability. Don't forget to         #
    # implement the L2 regularization!                                          #
    #############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    #need to get the dimensions we will be working with
    train = X.shape[0]
    y = y.reshape(train)
    scorelist=np.dot(X,W)
    expscorelist=np.exp(scorelist)
    expsums=np.sum(expscorelist,axis=1, keepdims=True)
    scoreprobabilities=expscorelist/expsums
    truth=-np.log(scoreprobabilities[range(train),y])
    loss=np.sum(truth)
    #Ridge regularization 
    loss= (loss/train)+((reg)*np.sum(W**2))/2
    dW=X.T.dot(scoreprobabilities)
    dW = (dW/train)+ ((reg)*np.sum(W**2))/2



    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    return loss, dW

In [ ]:
###################################################
# do not modify this cell
###################################################

def grad_check_sparse(f, x, analytic_grad, num_checks=10, h=1e-5):
    for i in range(num_checks):
        ix = tuple([randrange(m) for m in x.shape])
        oldval = x[ix]
        x[ix] = oldval + h  # increment by h
        fxph = f(x)  # evaluate f(x + h)
        x[ix] = oldval - h  # increment by h
        fxmh = f(x)  # evaluate f(x - h)
        x[ix] = oldval  # reset
        grad_numerical = (fxph - fxmh) / (2 * h)
        grad_analytic = analytic_grad[ix]
        rel_error = abs(grad_numerical - grad_analytic) / (abs(grad_numerical) + abs(grad_analytic))
        print(f'numerical: {grad_numerical} analytic: {grad_analytic}, relative error: {rel_error}')

In [ ]:
###################################################
# do not modify this cell
###################################################

# generating a random weight matrix and use it to compute the loss
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, Xdev, Ydev, 0.0)

# sanity check, our loss should be something close to -log(0.1).
print(f'loss: {loss}, compared to sanity check {-np.log(0.1)}\n')

# using numeric gradient checking as a debugging tool.
# the numeric gradient should be close to the analytic gradient.
f = lambda w: softmax_loss_naive(w, Xdev, Ydev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

# doing another gradient check with regularization
loss, grad = softmax_loss_naive(W, Xdev, Ydev, 5e1)
f = lambda w: softmax_loss_naive(w, Xdev, Ydev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

loss: [2.38801527], compared to sanity check 2.3025850929940455

numerical: [0.23978913] analytic: 0.2397891583901038, relative error: [6.76318286e-08]
numerical: [1.42659315] analytic: 1.4265931701953039, relative error: [5.78234457e-09]
numerical: [-1.34135702] analytic: -1.3413570859029003, relative error: [2.59018736e-08]
numerical: [-0.9618596] analytic: -0.9618596550289213, relative error: [2.91644555e-08]
numerical: [-1.54930296] analytic: -1.5493029878472, relative error: [1.01005986e-08]
numerical: [-4.29555286] analytic: -4.295552991509899, relative error: [1.47454358e-08]
numerical: [-1.2606551] analytic: -1.260655158210656, relative error: [2.41092806e-08]
numerical: [1.5272853] analytic: 1.5272853748866395, relative error: [2.30528418e-08]
numerical: [3.45149598] analytic: 3.4514958892037537, relative error: [1.34845269e-08]
numerical: [2.73607488] analytic: 2.7360748249676097, relative error: [1.02789425e-08]
numerical: [0.85745468] analytic: 0.8574545858411761, relative 

In [ ]:
###################################################
# do not modify this cell
###################################################

# checking that the vectorized version in softmax_loss_vectorized
# computes the same results. The vectorized version should be much faster.
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, Xdev, Ydev, 0.000005)
toc = time.time()
print(f'naive loss: {loss_naive} computed in {toc-tic} sec')

tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, Xdev, Ydev, 0.000005)
toc = time.time()
print(f'vectorized loss: {loss_vectorized} computed in {toc--tic} sec')

# use the Frobenius norm to compare the two versions of the gradient.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print(f'loss difference: {np.abs(loss_naive-loss_vectorized)}')
print(f'gradient difference: {grad_difference}')

naive loss: [2.34256238] computed in 26.34746813774109 sec
vectorized loss: 2.3425623794010395 computed in 3343131842.7717323 sec
loss difference: [0.]
gradient difference: 429.6270889875715


# If you are not working on the optional assignment, stop here

In [1]:
class Softmax(object):
    def __init__(self):
        self.W = None

    def train(self, X, Y, lr=1e-3, reg=1e-5, n_iters=100, b_size=200, verbose=False):
        """
        Train this linear classifier using stochastic gradient descent.

        Inputs:
        - X: A numpy array of shape (N, D) containing training data; there are N
          training samples each of dimension D.
        - Y: A numpy array of shape (N,) containing training labels; y[i] = c
          means that X[i] has label 0 <= c < C for C classes.
        - lr: (float) learning rate for optimization.
        - reg: (float) regularization strength.
        - n_iters: (integer) number of steps to take when optimizing
        - b_size: (integer) number of training examples to use at each step.
        - verbose: (boolean) If true, print progress during optimization.

        Outputs:
        A list containing the value of the loss function at each training iteration.
        """
        num_train, dim = X.shape
        num_classes = (np.max(Y) + 1)  
        if self.W is None: # initialize weights
            self.W = 0.001 * np.random.randn(dim, num_classes)

        # Run stochastic gradient descent to optimize W
        loss_history = []
        for it in range(n_iters):
            X_batch = None
            Y_batch = None
            #########################################################################
            # TODO: 20 points (optional)                                            #
            # Sample batch_size elements from the training data and their           #
            # corresponding labels to use in this round of gradient descent.        #
            # Store the data in X_batch and their corresponding labels in           #
            # Y_batch; after sampling X_batch should have shape (batch_size, dim)   #
            # and Y_batch should have shape (batch_size,)                           #
            #                                                                       #
            # Hint: Use np.random.choice to generate indices. Sampling with         #
            # replacement is faster than sampling without replacement.              #
            #########################################################################
            # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
            randomsamples=np.random.choice(num_train,batch_size,replace=False)
            X_batch=X[randomsamples]
            Y_batch=y[randomsamples]
            # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

            # evaluate loss and gradient
            loss, grad = self.loss(X_batch, Y_batch, reg)
            loss_history.append(loss)

            # perform parameter update
            #########################################################################
            # TODO: 20 points (optional)                                            #
            # Update the weights using the gradient and the learning rate.          #
            #########################################################################
            # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
            self.W-=grad*learning_rate
            # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

            if verbose and it % 100 == 0:
                print(f'iteration {it} / {n_iters}: loss {loss}')

        return loss_history

    def predict(self, X):
        """
        Use the trained weights of this linear classifier to predict labels for
        data points.

        Inputs:
        - X: A numpy array of shape (N, D) containing training data; there are N
          training samples each of dimension D.

        Returns:
        - y_pred: Predicted labels for the data in X. y_pred is a 1-dimensional
          array of length N, and each element is an integer giving the predicted
          class.
        """
        y_pred = np.zeros(X.shape[0])
        ########################################################################
        # TODO: 30 points                                                      #
        # Implement this method. Store the predicted labels in y_pred.         #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        scores=X.dot(W)
        y_pred=np.argmax(X.dot(self.W))
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        return y_pred

    def loss(self, bX, bY, reg):
        """
        Inputs:
        - bX: A numpy array of shape (N, D) containing a minibatch of N
          data points; each point has dimension D.
        - bY: A numpy array of shape (N,) containing labels for the minibatch.
        - reg: (float) regularization strength.

        Returns: A tuple containing:
        - loss as a single float
        - gradient with respect to self.W; an array of the same shape as W
        """
        return softmax_loss_vectorized(self.W, bX, bY, reg)

In [2]:
# results is dictionary mapping tuples of the form
# (learning_rate, regularization_strength) to tuples of the form
# (training_accuracy, validation_accuracy)
results = {}
# highest validation accuracy that we have seen so far.
best_val = -1
# Softmax object that achieved the highest validation rate.
best_softmax = None

################################################################################
# TODO:  30 points (optional)                                                  #
# Use the validation set to set the learning rate and regularization strength. #
# Save the best trained softmax classifer object in best_softmax.              #
#                                                                              #
# For each combination of hyperparameters, train a softmax classifier on the   #
# training set, compute its accuracy on the training and validation sets, and  #
# store these numbers in the results dictionary. In addition, store the best   #
# validation accuracy in best_val and the classifer that achieves this         #
# accuracy in best_softmax.                                                    #
#                                                                              #
# Hint: You should use a small number of iterations as you develop the         #
# validation code; once you are confident that your validation code works, you #
# should rerun the validation code using a larger number.                      #
#                                                                              #
# Note: you may see runtime/overflow warnings during hyper-parameter search. 
# This may be caused by extreme values, and is not a bug.
################################################################################

# provided as a reference. You may want (should) to change these hyperparameters
learning_rates = [1e-7, 5e-7]
regularization_strengths = [2.5e4, 5e4]

# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
pass
# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    
# Print out results.
for lr, reg in sorted(results):
    tr_acc, val_acc = results[(lr, reg)]
    print(f'lr {lr} reg {reg} train accuracy: {tr_acc} val accuracy: {val_acc}')
print(f'best validation accuracy achieved: {best_val}')

best validation accuracy achieved: -1


In [ ]:
###################################################
# do not modify this cell
###################################################

# evaluate the best softmax on test set
y_test_pred = best_softmax.predict(Xte)
test_accuracy = np.mean(Yte == y_test_pred)
print(f'softmax on raw pixels final test set accuracy: {test_accuracy}')

In [ ]:
###################################################
# do not modify this cell
###################################################

# visualize the learned weights for each class
w = best_softmax.W[:-1,:] # strip out the bias
w = w.reshape(32, 32, 3, 10)
w_min, w_max = np.min(w), np.max(w)
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(2, 5, i + 1)
    # rescale the weights to be between 0 and 255
    wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])